In [1]:
# Importing necessary libraries
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from IPython.display import HTML

styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
import random
import urllib
import re

# Create empty data frame with all interested variables
used_cars = pd.DataFrame(columns = ['year', 'make', 'model', 'body_type', 'doors', 'drivetrain',
             'engine', 'ext_color','combined_gas_mileage', 'fuel_type',
             'int_color', 'transmission', 'mileage','NHTSA_rating',
             'stock_number', 'VIN', 'market_diff', 'abv_bel_market', 'listed_price', 'price_assessment'])

# Run code until we get a dataset of at least 50,000 observations
while len(used_cars) <= 500: 
  # All car urls have the same format, but with a different 9 digit code between 300,000,000 and 400,000,000
  begin_url = 'https://www.cargurus.com/Cars/inventorylisting/viewDetailsFilterViewInventoryListing.action?zip=02138&distance=50000#listing=' 
  end_url = '/NONE'
  snapshot_url = begin_url + str(random.randrange(300000000,400000000)) + end_url

  # Run the requests functions and parse the html code
  snapshot = requests.get(snapshot_url)
  raw_html = snapshot.text
  soup = BeautifulSoup(raw_html, 'html.parser')

  # Get the information of listed price, difference of market average, and whether it is below or above market average
  market_compare_val = []
  market_compare_abv_bel = []
  for e in soup.find_all(class_="dealRatingDescription"):
    for t in list(e.stripped_strings):
      market_compare_val.append(t.split()[0])
      market_compare_abv_bel.append(t.split()[1])
  price_storage = []
  for i in soup.select('.price > span'):
    price_storage.append(str(i)[6:-7])
  price_assessment = re.findall(r'\w* DEAL', raw_html) # DOES NOT INCLUDE HIGH OR OVERPRICED MARKERS SO CANNOT USE

  # Looking at the majority of the car's information and bypassing cars with missing data
  car_node = soup.select('.propertyList')
  remove_list = []
  for i in range(len(car_node)):
    if len(car_node[i]) != 65: # Any value other than 65 represents missing variables
      remove_list.append(i)

  # Delete any cars that do not have the same number of predictors as we require
  for i in sorted(remove_list, reverse = True):
    del car_node[i]; del market_compare_val[i]; del market_compare_abv_bel[i]

  # Separate the cars in the list that a single page gives and separate the variable data
  for i in range(len(car_node)):
    car_data = list(soup.select('.propertyList')[i])[3::4] # get all essential car data
    cleaned_car_data = []
    for j in car_data:
      cleaned_car_data.append(str(j)[4:-6])
    # Add the additional variables from before to list
    cleaned_car_data.append(market_compare_val[i])
    cleaned_car_data.append(market_compare_abv_bel[i])
    cleaned_car_data.append(price_storage[i])
    cleaned_car_data.append(price_assessment[i])

    # Sometimes the order of the variables are changed, so we check that safety rating is where it should, otherwise we do not add to dataset
    if (cleaned_car_data[13] in ['Not Rated', '1', '2', '3', '4', '5']) and (13 < len(cleaned_car_data)):
      used_cars = used_cars.append(pd.Series(cleaned_car_data, index = used_cars.columns[:len(cleaned_car_data)]), ignore_index = True)
    
  # While the probability of having duplicate urls is extremely small, safe to remove possible duplicates
  used_cars = used_cars.drop_duplicates(keep = 'first')
used_cars

,year,make,model,body_type,doors,drivetrain,engine,ext_color,combined_gas_mileage,fuel_type,int_color,transmission,mileage,NHTSA_rating,stock_number,VIN,market_diff,abv_bel_market,listed_price,price_assessment
0,2022,Toyota,Highlander,SUV / Crossover,4 doors,All-Wheel Drive,295 hp 3.5L V6,Red,23 MPG,Gasoline,Black,8-Speed Automatic,"12,609",5,N15823C,5TDGZRBH8NS165214,"$1,443",below,"$41,000",GOOD DEAL
1,2018,Ford,Explorer,SUV / Crossover,4 doors,All-Wheel Drive,290 hp 3.5L V6,Silver,19 MPG,Gasoline,Ebony Black,6-Speed Automatic,"92,947",5,P3525B,1FM5K8F87JGB33404,"$1,706",below,"$22,975",GOOD DEAL
2,2019,Mazda,CX-5,SUV / Crossover,4 doors,All-Wheel Drive,187 hp 2.5L I4,Red,27 MPG,Gasoline,Black,6-Speed Automatic,"49,300",5,FT22220A,JM3KFBDM1K1556158,"$2,651",below,"$24,900",GREAT DEAL
3,2021,Volvo,S60,Sedan,4 doors,All-Wheel Drive,316 hp 2L I4,Red,26 MPG,Gasoline,Blond,Automatic,"18,335",Not Rated,IP7230,7JRA22TL6MG096103,"$2,401",below,"$36,900",GREAT DEAL
4,2018,Volvo,XC90,SUV / Crossover,4 doors,All-Wheel Drive,316 hp 2L I4,Gray,23 MPG,Gasoline,Black (Charcoal),Automatic,"44,952",Not Rated,CR10080FD,YV4A22PL4J1373058,"$2,019",below,"$37,500",GREAT DEAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2019,Audi,Q5,SUV / Crossover,4 doors,All-Wheel Drive,248 hp 2L I4,Gray,24 MPG,Gasoline,Brown (BEIGE),7-Speed Dual Clutch,"23,579",5,111506,WA1BNAFY6K2111506,"$2,018",below,"$37,883",GREAT DEAL
500,2020,Subaru,Forester,SUV / Crossover,4 doors,All-Wheel Drive,182 hp 2.5L H4,Silver,29 MPG,Gasoline,Gray,Continuously Variable Transmission,"30,736",5,S16575A,JF2SKARC9LH561150,$723,below,"$28,492",GOOD DEAL
501,2021,Subaru,Outback,SUV / Crossover,4 doors,All-Wheel Drive,182 hp 2.5L H4,Black,29 MPG,Gasoline,Slate Black,Continuously Variable Transmission,"22,741",Not Rated,SC1132,4S4BTADC8M3100499,$958,below,"$27,984",GOOD DEAL
502,2015,Audi,A4,Sedan,4 doors,All-Wheel Drive,220 hp 2L I4 Flex Fuel Vehicle,Blue,25 MPG,Flex Fuel Vehicle,Black,8-Speed Automatic,"91,652",5,M116021,WAUFFAFL0FN007457,"$2,545",below,"$14,223",GREAT DEAL


In [4]:
# Save data frame to csv on drive
from google.colab import drive
drive.mount('drive')
used_cars.to_csv('used_cars.csv')
!cp used_cars.csv "drive/My Drive/"

Mounted at drive
